# What is this application doing?

In the below program, I am reading in a subset of LinkedIn posts by a given author, filtering down to the top X number of most upvoted posts (sorted in descending order). I then take that context, and ask OpenAI (using GPT4 for now, but may see if newest version of GPT4 does any better than the old one, old one, sheesh that's funny to say) to evaluate it and judge style/tone/etc. GPT comes back with 25 ways to measure it, and I then ask GPT4 to evaluate the 25 posts across those 25 properties, to sinmulate to the best of it's ability, the user in question, and write a short trivial post about eating a sandwich in the park. 

### Installing Required Packagages

In [ ]:
%pip install langchain python-dotenv openai

### Importing Required Libraries

In [43]:
# LangChain
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate

# Environment Variables
import os

# Loading of environment variables (API keys, etc.)
from dotenv import load_dotenv

# LinkedIn requirements
import requests
import json

load_dotenv()

True

### Setting our OpenAI API Key

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')

### Creating our LLM Object

In [28]:
llm = ChatOpenAI(temperature=0, openai_api_key=openai_api_key, model_name='gpt-4', client="my_client")


### Definining our Function to get User Posts from LinkedIn

In the below cell, I used RapidAPI to featch LinkedIn posts by profile, you could easily swap out something related to Twitter/X, or even substitute reading from a file. Replace the code below with whatever makes sense for you. 

In [45]:
def get_users_posts(screen_name, posts_to_return):
    url = "https://fresh-linkedin-profile-data.p.rapidapi.com/get-profile-posts"

    querystring = {"linkedin_url": f"https://www.linkedin.com/in/jakemakler/", "type": "posts"}   # replace with your LinkedIn URL
    
    headers = {
	"X-RapidAPI-Key": os.getenv('X_RAPIDAPI_KEY'), # replace with your API key
	"X-RapidAPI-Host": os.getenv('X_RAPIDAPI_HOST') # replace with your host
    }

    response = requests.get(url, headers=headers, params=querystring)

    # load the data into a Python dictionary
    data = json.loads(response.text)
    print(response.text)

    # access the 'data' field (a list of posts)
    posts = data['data']

    # sort posts by 'num_likes' in descending order
    sorted_posts = sorted(posts, key=lambda post: post['num_likes'], reverse=True)

    # get the 'text' field of the top 100 posts
    texts = [post['text'] for post in sorted_posts[:posts_to_return]]   

    # join the texts into a single string separated by two new lines
    example_posts = '\n\n'.join(texts)
    return example_posts

### Calling `get_user_posts` for our Target User

In [ ]:
user_screen_name = 'testuser'  # Replace this with the desired user's screen name
example_posts = get_users_posts(user_screen_name, 25) # change the # of example posts to grab
print(example_posts)

### Our First LLM Template

This is just trying to mimic without first evaluating the users style, just matching (first level of effort).

In [ ]:
template = """
Please create me a LinkedIn post about going to the park and eating a sandwich.

% TONE
 - Don't use any emojis or hashtags.
 - Respond in the tone of <insert name>

% START OF EXAMPLE POSTS TO MIMIC
{example_posts}
% END OF EXAMPLE POSTS TO MIMIC

YOUR POST:
"""

prompt = PromptTemplate(
    input_variables=["example_posts"],
    template=template,
)

final_prompt = prompt.format(example_posts=example_posts)

print (final_prompt)

### Let's Dig Deeper

Asking GPT4 to generate a list of style and tone attributes

In [ ]:
prompt = """
Can you please generate a list of tone attributes and a description to describe a piece of writing by?

Things like pace, mood, etc.

Respond with nothing else besides the list
"""

how_to_describe_tone = llm.predict(prompt)
print (how_to_describe_tone)

### Our Function to Get the Authors Tone

This function takes in the list of attributes from above, and combines it with our example posts from above, to evaluate the authors tone and style, (level 2 effort) to better mimic the user.

In [22]:
def get_authors_tone_description(how_to_describe_tone, example_posts):
    template = """
        You are an AI Bot that is very good at generating writing in a similar tone as examples.
        Be opinionated and have an active voice.
        Take a strong stance with your response.

        % HOW TO DESCRIBE TONE
        {how_to_describe_tone}

        % START OF EXAMPLES
        {example_posts}
        % END OF EXAMPLES

        List out the tone qualities of the examples above
        """

    prompt = PromptTemplate(
        input_variables=["how_to_describe_tone", "example_posts"],
        template=template,
    )

    final_prompt = prompt.format(how_to_describe_tone=how_to_describe_tone, example_posts=example_posts)

    authors_tone_description = llm.predict(final_prompt)

    return authors_tone_description

### Calling our new Function

In [ ]:
authors_tone_description = get_authors_tone_description(how_to_describe_tone, example_posts)
print (authors_tone_description)

### This is where things get interesting

In this block we are taking it to level 3 effort, mixing our tone/style descriptions, example posts, and some specific prompting to get us the "ultimate" ghost writer. 

In [24]:
template = """
% INSTRUCTIONS
 - You are an AI Bot that is very good at mimicking an authors writing style.
 - Your goal is to write content with the tone that is described below.
 - Do not go outside the tone instructions below
 - Do not use hashtags or emojis
 - Respond in the tone of Jake Makler

% Description of the authors tone:
{authors_tone_description}

% Authors writing samples
{example_posts}

% YOUR TASK
Please create a LinkedIn about going to the park and eating a sandwich.
"""

prompt = PromptTemplate(
    input_variables=["authors_tone_description", "example_posts"],
    template=template,
)

final_prompt = prompt.format(authors_tone_description=authors_tone_description, example_posts=example_posts)

In [ ]:
llm.predict(final_prompt)